# Review: β-VAE: Learning Basic Visual Concepts with a Constrained Variational Framework

## Paper Reviews

### Prior Reseach

+ **disentanglement**는 latent representation space를 이용하는 generative model 전반에서 중요한 화두임
+ 이는 우리가 다루는 latent space는 직관적으로 **interpretable한 space가 아니기 때문**
+ 따라서 실제로 generate하는 interpretable한 대상의 **factor**를 결정하는 feature가 직접적으로 latent vector의 dimension axis와 대응되지 않을 수 있음( = **entangled**)
+ 그러나 분명히 우리는 추상적으로 해당 factor를 선형적이게 catch 할 수 있기 때문에 해당 representation 또한 선형으로 표현될 수 있다는 것을 기대할 수 있음
+ 그렇다면 이를 달성할 수 있도록 model을 regularize하여 **latent space의 각 axis를 interpretable하게 달성할 수도 있지 않을까?**

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/167542507-bddcea30-4710-4198-a281-337b9e17bf3c.png" alt="1" width="400px" />
</p>

+ 이러한 목적을 달성하기 위해서 unsupervised method로는 **InfoGAN**가 제시되었음
+ InfoGAN은 Generator와의 **Mutual Information**을 maximize하는 latent code $c$를 새로 도입하는 방식으로 이를 실현했음
+ 그러나 기본적으로 GAN Architecture에 의존하기 때문에 학습이 불안정하고 prior distribution에 민감하여 prior knowledge에 의존한다는 점에서 개선이 필요함

$$ \min_{G, Q}\max_{D}V_{\text{InfoGAN}}(D,G,Q)=V(D,G)−λL_I(G,Q) $$

+ 따라서 이러한 문제를 개선하기 위해 VAE Architecture에서 개선을 가한 **β-VAE**를 제시
+ β-VAE는 InfoGAN가 catch하지 못하는 latent factor를 학습하면서 disentanglement를 잘 달성할 수 있었음

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/167542516-ab56fba3-20f8-4f58-9739-de85c12bc00e.png" alt="2" width="400px" />
</p>



### Derivation of β-VAE Objective

+ $\mathbf{x} \in X$ = images, $\mathbf{v} \in V, \mathbf{w} \in W$ = factors로 구성된 dataset $\mathcal{D} = \{X, V, W\}$을 가정
+ $\mathbf{v} \in \mathbb{R}^K$는 conditionally independent factor로서 $\log p(\mathbf{v}|\mathbf{x})=\sum_k \log p(v_k|\mathbf{x})$를 만족하며 $\mathbf{w} \in \mathbb{R}^H$는 conditionally dependent factor
+ 우리가 generative model로서 원하는 것은 $M \ge K$인 $\mathbf{z} \in \mathbb{R}^M$으로 $p(\mathbf{x}|\mathbf{z}) \approx p(\mathbf{x}|\mathbf{v}, \mathbf{w})$를 달성하는 것
+ 따라서 기본적으로 만족시켜야 할 generative model의 특징은 다음과 같이 marginal likelihood를 최대화시켜야 함

$$ \max_{\theta} \mathbb{E}_{p_{\theta}(\mathbb{z})} [p_{\theta}(\mathbf{x}|\mathbf{z})] $$

+ 이러한 좋은 $\mathbf{z}$를 찾기 위해 observation $\mathbf{x}$로 posterior distribution $q_{\phi}(\mathbf{z}|\mathbf{x})$를 이용
+ 이 때 우리가 추가적으로 원하는 것은 $q_{\phi}(\mathbf{z}|\mathbf{x})$가 $\mathbf{v}$를 disentangle하게 catch하고 남은 $\mathbf{z}$의 subset으로 $\mathbf{w}$를 표현할 수 있도록 만들어야 함
+ 따라서 $q_{\phi}(\mathbf{z}|\mathbf{x})$를 prior $p(\mathbf{z}) \sim \mathcal{N}(\mathbf{0}, I)$로 match하는 과정에서 **constraint** 도입
+ 이를 통해서 **latent informational bottleneck**의 capacity를 조절하여 **independent**한 $\mathbf{v}$를 catch 할 수 있음
+ 이러한 constraint를 위해서 $\epsilon > 0$을 도입하여 objective를 다음과 같이 표현 가능

$$ \max_{\theta, \phi} \mathbb{E}_{\mathbf{x} \sim \mathcal{D}} [\mathbb{E}_{q_{\phi}(\mathbf{z}|\mathbf{x})} [\log p_{\theta} (\mathbf{x} | \mathbf{z})] ]\,\,\,\text{subject to}\,\,\, D_{KL}(q_{\phi}(\mathbf{z}|\mathbf{x}) || p(\mathbf{z}) < \epsilon $$

+ 이를 **KKT condition**을 통해서 multiplier $\beta > 0$를 도입하면 **complementary slackness**($ \beta \cdot \epsilon = 0$)를 통해서 다음과 같이 objective를 표현 가능

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/167542521-b299e9d3-e4c8-4a5b-9190-ed99e98583c1.png" alt="3" width="400px" />
</p>


$$ \max_{\theta, \phi} \mathbb{E}_{\mathbf{x} \sim \mathcal{D}} [\mathbb{E}_{q_{\phi}(\mathbf{z}|\mathbf{x})} [\log p_{\theta} (\mathbf{x} | \mathbf{z})] ] - \beta ({D_{KL}(q_{\phi}(\mathbf{z}|\mathbf{x}) || p(\mathbf{z})})) $$

+ 이러한 objective를 사용하는 VAE를 **β-VAE**라고 할 수 있으며 $\beta = 1$인 경우는 vanilla VAE와 동일한 objective가 됨
+ 만약 $ \beta > 1$이라면 해당 objective는 $\mathbf{z}$가 표현할 수 있는 **capacity를 제한**하게 되며 이는 likelihood를 최대화하는 것과 함께 진행되어 disentanglement가 증가될 수 있음
+ 실제로 disentanglement의 달성을 저자들이 도입한 **disentanglement metric score**를 통해서 확인이 가능함!

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/167542525-642bb09d-7823-42f3-9c93-0fab85eef74c.png" alt="4" width="500px" />
</p>

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/167542530-c0ab9247-0b81-452e-8dad-71539d74d3a2.png" alt="5" width="500px" />
</p>

### How β-VAE Achieved Disentanglement? ("Understanding Disentangling in β-VAE")


+ β-VAE가 disentanglement를 달성할 수 있는 원리는 **information bottleneck**과 깊은 연관이 있음
+ information bottleneck은 task $Y$를 input $X$로 표현하기 위해서 **최대한으로 압축된 bottleneck signal $Z$를 찾는 문제**
+ 이 것은 $Z$와 $Y$의 mutual information을 최대화하면서 $Y$를 표현하기 위해 불필요한 $X$를 최소화하는 것으로 달성될 수 있음

$$  \max [I(Z; Y) - \beta I(X; Z)] $$

+ 이러한 information bottleneck으로 β-VAE objective가 어떻게 latent representation $\mathbf{z}$에 작용하여 disentanglement를 달성하는지 해석이 가능
+ β-VAE에서의 **posterior $q_{\phi}(\mathbf{z}|\mathbf{x})$는 reconstruction task $\mathbb{E}_{q (\mathbf{z}|\mathbf{x})} [\log p (\mathbf{x} | \mathbf{z})]$를 위한 information bottleneck**이 되는 것
+ 이러한 information bottleneck으로 **$\mathbf{x}$의 정보를 효율적인 압축으로 전달하기 위해서** $\beta$로 조절되는 prior $p(\mathbf{z})$와의 KL divergence term을 constraint를 사용하는 것

$$ \max_{\theta, \phi} \mathbb{E}_{\mathbf{x} \sim \mathcal{D}} [\mathbb{E}_{q_{\phi}(\mathbf{z}|\mathbf{x})} [\log p_{\theta} (\mathbf{x} | \mathbf{z})] ] - \beta ({D_{KL}(q_{\phi}(\mathbf{z}|\mathbf{x}) || p(\mathbf{z})})) $$

+ VAE에서 prior $p(\mathbf{z})$는 dimension마다 unit Gaussian $ p(z_i) \sim \mathcal{N}(0, 1)$로 표현될 수 있기 때문에 **posterior $q_{\phi}(\mathbf{z}|\mathbf{x})$ 또한 diagonal covariance matrix**가 됨
+ 따라서 각각의 latent channel $z_i$는 independently하게 input $x_n$의 information을 전달하는 역할을 수행
+ 이는 **KL divergence term이 각 channel 별 전달할 수 있는 정보량의 upper bound**로 기능하게 된다는 것을 의미함
+ KL divergence term은 $ p(z_i|\mathbf{x}) = p(\mathbf{z}) $일 때 0이 되는데, 이는 $ \mu_i = 0 $, $ \sigma_i = 1 $를 의미하므로 각 channel의 capacity가 0이 되어야 최소화 된다는 것
+ 이러한 capacity를 늘리기 위해서는 posterior mean을 떨어트리거나 posterior variance를 늘리는 방법 밖에 없는데, **이는 반드시 KL divergence term의 증가를 가져오게 됨**


+ 따라서 이렇게 KL divergence term을 유지하기 위해서는 posterior mean을 밀집시키거나 posterior variance를 크게 만드는 수 밖에 없음
+ 그러나 이러한 결과는 다른 data point에서 오는 posterior와 더 **overlap**하게 만들기 때문에 reconstruction에서 어떠한 sample에서 온 것인지 confusion이 쉽게 발생 가능함
+ 이것은 결과적으로 posterior에서 **real sample $x_2$에 대해서 different sample $x_1$과의 구분이 어렵기 때문에 likelihood cost $\mathbb{E}_{q(\mathbf{z}|\mathbf{x})} [\log p (\mathbf{x} | \mathbf{z})]$의 증가**를 가져오게 됨

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/167542531-42c709b4-71c6-466e-bd6d-b0c8cdbbff9b.png" alt="6" width="500px" />
</p>

+ 만약 objective에서 이러한 bottleneck에 대한 pressure가 강하게 이루어지지 않는다면(i.e. vanilla VAE), 이러한 posterior의 locality property가 강하게 달성되지 않음
+ 따라서 이러한 pressure를 강하게 주는 낮은 $\beta$에서는 latent space reconstruction이 쉽게 되지만 pressure를 강하게 주는 높은 $\beta$에서는 reconstruction에서 어려워짐

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/167542532-f1932d78-5217-45b2-aec0-45d170978a68.png" alt="7" width="500px" />
</p>



+ 하지만 high bottleneck pressure에 대해서 reconstruction이 어려운 것은 확인할 수 있어도 **왜 disentanglement가 더 달성될 수 있었는가**는 다른 문제임
+ 이러한 disentanglement의 원인에 대해서 저자들은 우리가 인지할 수 있는 **factor마다 reconstruction에서의 중요도가 다르기 때문**이라 주장함
+ 예를 들어 position에 대한 information은 shape 같은 factor에 비해서 변할 때 pixel의 변화가 훨씬 크기 때문에 reconstruction cost에 영향을 주는 power가 강함
+ 따라서 bottleneck pressure가 존재한다면 최대한 먼저 position에 대한 정보를 담고 난 이후 나머지 factor의 information을 담도록 유도됨


+ 그러나 이러한 factor마다의 중요도가 어떻게 각 latent channel에 선형적으로 대응되는지(= align with factor)에 대해서 이러한 접근은 바로 설명이 되지 않음
+ 이에 대해서 저자들은 다음과 같은 이유로 factor의 중요도가 disentanglement를 달성하는지 추론함

1. posterior locality pressure가 존재하는 상황에서 새로운 dimension에 factor information을 담는 것이 **기존에 더 중요했던 factor information을 해치지 않는 자연스러운 방향**임
2. 따라서 model은 각 factor를 encoding하는 axis에 할당되는 average capacity를 조절하여(= posterior의 variance를 조정하여) reconstruction cost를 최소화하려고 유도됨
3. 그러나 **posterior는 diagonal covariance를 만족해야 하기 때문**에 위와 같은 방법이 불가능하게 되므로 각 channel마다 factor의 information을 담도록 유도될 수 밖에 없음!

+ 만약 위와 같은 가설이 사실이라면, KL divergence term에 새로운 **capacity parameter** $C$를 도입해서 변화하는 양상을 check 해볼 수 있음
+ 이렇게 접근해보면 추측과 같이 아래와 같이 position에 해당하는 x, y factor가 먼저 encoding되고 다른 factor가 남은 영역에 encoding되는 것을 확인할 수 있음

$$ \mathcal{L}(\theta, \phi; \mathbf{x}, \mathbf{z}, C) = \mathbb{E}_{q_{\phi}(\mathbf{z}|\mathbf{x})} [\log p_{\theta} (\mathbf{x} | \mathbf{z})] - \gamma | D_{KL}(q_{\phi}(\mathbf{z}|\mathbf{x}) || p(\mathbf{z})) - C |$$

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/167542535-891fafab-f713-4eb2-9b7d-8a9127d5392f.png" alt="8" width="500px" />
</p>




+ 가설을 다시 고려해보면, 만약 이러한 capacity $C$가 낮은 상황에서는 강한 bottleneck이 발생하여 disentanglement가 발생하지만 많은 information을 담기 어려움
+ 따라서 이러한 $C$를 **training 중에 적절히 증가시키는** 방법을 통해서 좋은 reconstruction과 disentanglement를 동시에 달성할 수도 있음

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/167542538-801ea298-be2b-48c6-91c5-581195afa7b0.png" alt="9" width="500px" />
</p>

## Implementation Reviews

+ paper의 중요 contribution인 [objective](https://github.com/AntixK/PyTorch-VAE/blob/master/models/beta_vae.py) implementation
+ ```self.loss_type```이 ```B```인 경우가 capacity parameter를 추가로 도입하는 경우


In [ ]:
def loss_function(self,
                      *args,
                      **kwargs) -> dict:
        self.num_iter += 1
        recons = args[0]
        input = args[1]
        mu = args[2]
        log_var = args[3]
        kld_weight = kwargs['M_N']  # Account for the minibatch samples from the dataset

        recons_loss =F.mse_loss(recons, input)

        kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp(), dim = 1), dim = 0)

        if self.loss_type == 'H': 
            loss = recons_loss + self.beta * kld_weight * kld_loss
        elif self.loss_type == 'B': 
            self.C_max = self.C_max.to(input.device)
            C = torch.clamp(self.C_max/self.C_stop_iter * self.num_iter, 0, self.C_max.data[0])
            loss = recons_loss + self.gamma * kld_weight* (kld_loss - C).abs()
        else:
            raise ValueError('Undefined loss type.')

        return {'loss': loss, 'Reconstruction_Loss':recons_loss, 'KLD':kld_loss}

## Reference

https://openreview.net/pdf?id=Sy2fzU9gl  
https://arxiv.org/pdf/1804.03599.pdf  
https://github.com/AntixK/PyTorch-VAE/blob/master/models/beta_vae.py